In [ ]:
from tqdm.auto import tqdm
import sleep
import pandas as pd
import csv
import numpy as np
import random
import transformers
from transformers import DistilBertModel, DistilBertConfig, Di

## Load Data

In [ ]:
df_train = pd.read_pickle("data/processed/df_train_before_embedding.pkl")

In [ ]:
#Preprocess sentences
# Lower Case
df_train['sentence'] = df_train['sentence'].str.lower()
# Replace punctuations
df_train['sentence'] = df_train['sentence'].str.replace('[^\w\s]','')

In [ ]:
#convert data to list
list_sentence = df_train['sentence'].tolist()

## Load Model

In [ ]:
base_model_name = "distilbert-base-cased"  # cased as in benchmarking paper
tokenizer = DistilBertTokenizer.from_pretrained(base_model_name)
model = DistilBertModel.from_pretrained(base_model_name)

## Prepare Batches

In [ ]:
batch_sentence = []
for i in range(0, len(list_sentence), 8):
  batch_sentence.append(list_sentence[i:i +8])

## Tokenize

In [ ]:
encoded_input = []
for batch in tqdm(batch_sentence):
   encoded_input.append(tokenizer(batch, truncation = True, padding = True, return_tensors = "pt"))

## Create Embeddings

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

In [ ]:
last_hidden_states = []

for batch in tqdm(encoded_input):
  with torch.no_grad():
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    last_hidden_states.append(model(input_ids = input_ids, att

In [ ]:
torch.save(last_hidden_states, "data/processed/bert_base_case_embeddings.ckpt")